In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('digital.xlsx', sheet_name='Cleaning')

In [3]:
import re

# Function to extract numeric value
def extract_amount(val):
    # Find first number (handles decimals), even if text is mixed (e.g. "SGD 1200a")
    match = re.search(r'[\d,.]+', str(val))
    if match:
        # Remove commas, convert to float
        return float(match.group().replace(",", ""))
    else:
        return None

# Function to extract currency code
def extract_currency(val):
    # Look for known currency codes or symbol
    val = str(val)
    if "USD" in val: return "USD"
    if "TWD" in val: return "TWD"
    if "SGD" in val: return "SGD"
    if "AUD" in val: return "AUD"
    if "$" in val: return "USD"
    if "CAD" in val: return "CAD"
    if "INR" in val: return "INR"
    if "PHP" in val: return "PHP"
    if "GBP" in val: return "GBP"
    if "VND" in val: return "VND"
    # Add more currencies if needed
    return None

# Apply functions to the Spend column
df['Spend_Amount'] = df['Spend'].apply(extract_amount)
df['Spend_Currency'] = df['Spend'].apply(extract_currency)

# Fill missing currency from Region default (example for Australia)
region_currency = {
    "US": "USD", "CA": "USD", "UK": "USD", "AU": "AUD", "IN": "USD", "PH": "USD",
    "SG": "SGD", "VN": "USD", "TW": "TWD"
}

df['Spend_Currency'] = df.apply(
    lambda row: row['Spend_Currency'] if pd.notnull(row['Spend_Currency'])
    else region_currency.get(row['Region'], None), axis=1
)


In [4]:
from datetime import datetime

# List all your possible date formats
date_formats = [
    "%Y/%m/%d",     # 2025/12/27
    "%Y.%m.%d",     # 2023.03.09
    "%Y-%m-%d",     # 2025-12-04
    "%b %d, %Y",    # Apr 24, 2023
    "%m/%d/%Y",     # 09/23/2023
]

def parse_flexible_date(val):
    for fmt in date_formats:
        try:
            return datetime.strptime(str(val).strip(), fmt)
        except:
            continue
    return pd.NaT

# Apply to your DataFrame's Date column
df['Date_Clean'] = df['Date'].apply(parse_flexible_date)

# Now you can filter or keep only valid dates, as needed
# Example: Only keep rows with valid dates
df = df[df['Date_Clean'].notna()]


In [5]:
exchange_rates = {
    "USD": 1,
    "SGD": 0.74,
    "TWD": 0.031,
    "CAD": 0.73,
    "GBP": 1.28,
    "INR": 0.012,
    "PHP": 0.017,
    "VND": 0.000039,
    "AUD": 0.66      
}


In [6]:
import numpy as np

def convert_to_usd(amount, currency):
    try:
        rate = exchange_rates.get(currency, None)
        if rate is None or pd.isna(amount):
            return np.nan  # or None
        return amount * rate
    except Exception:
        return np.nan

df['Spend_USD'] = df.apply(lambda row: convert_to_usd(row['Spend_Amount'], row['Spend_Currency']), axis=1)

df['Spend_USD'] = df['Spend_USD'].round(2)


In [7]:
import re

# Function to extract numeric value
def extract_amount(val):
    match = re.search(r'[\d,.]+', str(val))
    if match:
        return float(match.group().replace(",", ""))
    else:
        return None

# Function to extract currency code
def extract_currency(val):
    val = str(val)
    for curr in exchange_rates.keys():
        if curr in val:
            return curr
    if "$" in val:
        return "USD"
    return None

df['Revenue_amount'] = df['Revenue'].apply(extract_amount)
df['Revenue_currency'] = df['Revenue'].apply(extract_currency)

# Fill missing currency based on Region if blank
region_currency = {
    "US": "USD", "CA": "USD", "UK": "USD", "AU": "AUD", "IN": "USD", "PH": "USD",
    "SG": "SGD", "VN": "USD", "TW": "TWD"
}

df['Revenue_currency'] = df.apply(
    lambda row: row['Revenue_currency'] if pd.notnull(row['Revenue_currency'])
    else region_currency.get(row['Region'], None), axis=1
)


In [8]:
def convert_to_usd(amount, currency):
    try:
        rate = exchange_rates.get(currency, None)
        if rate is None or pd.isna(amount):
            return None
        return amount * rate
    except Exception:
        return None

df['Revenue_USD'] = df.apply(lambda row: convert_to_usd(row['Revenue_amount'], row['Revenue_currency']), axis=1)


In [9]:
# save the cleaned DataFrame to a new Excel file
df.to_excel('cleaned_data.xlsx', index=False)